# Notebook to VI rotation curve galaxies

In [ ]:
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle

In [ ]:
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/n/nravi/legacysurvey_cache/' # change this path
vi_dir = '/pscratch/sd/n/nravi/vi_rot_curve_targets/' # change this path

In [ ]:
def get_cutout(SGA_ID, ra, dec, size, cache_dir, verbose=False):
    '''Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    SGA_ID : int
        SGA ID for galaxy.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    size : float
        size of cutout.
    cache_dir : string
        cache location
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    w : astropy.wcs.WCS
        World coordinate system for the image.
    '''
    
    # Either load an existing image or download a cutout.
    img_name = cache_dir + '{}.jpg'.format(SGA_ID)
    
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))

    else:
        img_url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&zoom=14&layer=ls-dr9&size={}&sga'.format(ra, dec, size)
        if verbose:
            print('Get {}'.format(img_url))
            
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
                
    # Set up the WCS.
    wcs_input_dict = {
        'CTYPE1': 'RA---TAN',
        'CUNIT1': 'deg',
        'CDELT1': -0.262/3600,
        'CRPIX1': size/2 + 0.5,
        'CRVAL1': ra,
        'NAXIS1': size,
        'CTYPE2': 'DEC--TAN',
        'CUNIT2': 'deg',
        'CDELT2': 0.262/3600,
        'CRPIX2': size/2 + 0.5,
        'CRVAL2': dec,
        'NAXIS2': size
    }
    w = WCS(wcs_input_dict)
    
    return img_name, w

In [ ]:
# load fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA.fits')
tf_fuji[:5]

In [ ]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA.fits')
tf_iron[:5]

In [ ]:
# load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i
SGA[:5]

In [ ]:
# list of rotation curve galaxies
tf_mastertable = vstack([tf_fuji,tf_iron])
rot_curve_gals = np.unique(tf_mastertable['SGA_ID'][tf_mastertable['ROT_CURVE']==1])

In [ ]:
# for each rotation curve galaxy, grab cut out, draw fibers on image, and save image
for sga_id in rot_curve_gals:
    
    targ_list = tf_mastertable[tf_mastertable['SGA_ID'] == sga_id]
    if np.min(targ_list['SKY_FIBER_DIST_R26']) < 0.001:
        center_target = targ_list[np.argmin(targ_list['SKY_FIBER_DIST_R26'])]['TARGETID']
        z = targ_list[np.argmin(targ_list['SKY_FIBER_DIST_R26'])]['Z']
        
    else:
        continue
    
    ra, dec = float(SGA['RA'][SGA_dict[sga_id]]), float(SGA['DEC'][SGA_dict[sga_id]])
    
    # D26 in arcmin
    d26 = SGA['D26'][SGA_dict[sga_id]]
    

    npix = np.minimum(int(2 * d26*60/0.262), 512)
    

    img_file, wcs = get_cutout(sga_id, ra, dec, size=npix, cache_dir,verbose=True)
    img = mpl.image.imread(img_file)

    fig1 = plt.figure(figsize=(7,5))

    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'SGA_ID: {}\n$z={{{:.4f}}}$'.format(sga_id, z), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');

    # Add the location of the DESI fibers.
    # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
    r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='black', facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'))
    r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='red', facecolor='none', alpha=0.8, lw=2,
                         transform=ax.get_transform('icrs'))
    ax.add_patch(r1)
    ax.add_patch(r2)

    for targ in targ_list:
        ra, dec = targ['RA'], targ['DEC']
        
        edgecolor2 = 'orange'

        # Add the location of the DESI fibers.
        # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor='lightcoral', facecolor='none', alpha=1, lw=3,
                             transform=ax.get_transform('icrs'))
        r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor=edgecolor2, facecolor='none', alpha=0.8, lw=2,
                             transform=ax.get_transform('icrs'))
        ax.add_patch(r1)
        ax.add_patch(r2)
        
        ax.text(ra, dec, str(targ['TARGETID']), transform=ax.get_transform('icrs'), color='white', fontsize=6)
    
    fig1.subplots_adjust(top=0.85, right=0.85, bottom=0.15, left=0.15)
    
    fig1.savefig(vi_dir + '{}.png'.format(sga_id), dpi=120)
    
    fig1.clear()
    plt.close(fig1)


In [ ]:
# we want to visually inspect rotation curve galaxies that have EOA targets
EOA_gals = np.unique(tf_mastertable[np.logical_and(tf_mastertable['EOA'] == 1, tf_mastertable['ROT_CURVE']==1)]['SGA_ID'])
print(EOA_gals)

In [ ]:
# inspect all the necessary galaxies, 
# if you find a galaxy with bad target for rotation curve analysis, grab all observations for that galaxy
tf_mastertable[tf_mastertable['SGA_ID'] == ??? ]
# identify which target is bad by target id and add to the list in the cell below

In [ ]:
# if there is a target not suitable for rotation curve analysis,
#change 'ROT_CURVE' to 0 for that target in the tables 
# (targets can be in both fuji and iron)

targetids = []

for targetid in targetids:
    try:
        tf_fuji['ROT_CURVE'][tf_fuji['TARGETID'] == targetid] = 0
    except:
        print(targetid, ' not in fuji')
    
    try:
        tf_iron['ROT_CURVE'][tf_iron['TARGETID'] == targetid] = 0
    except:
        print(targetid, ' not in iron')

In [ ]:
# additional targets that need to inspected, some of these may be elliptical galaxies which we can't use
# for rotation curve analysis
EXT_centers = tf_mastertable[np.logical_and(tf_mastertable['PVTYPE'] == 'EXT', tf_mastertable['SKY_FIBER_DIST_R26'] < 0.001)]
EXT_centers[EXT_centers['ROT_CURVE'] == 1]